# <center> <b> <font color='blue'> Metrics </center> </b> </font>

## <font color='#1f618d'> Table of Contents </font>

1. [Introduction](#1)
2. [Setup](#2)
3. [Predictions](#3)
4. [Other ways to forecast](#4)
5. [Summary](#5)
6. [References](#References)

<a name="1"></a>
## <font color='#1f618d'> <b> 1. Introduction </font> </b>

The goal is to show how to calculate metrics for a time series problem.

<a name="2"></a>
## <font color='#1f618d'> <b> 2. Setup </font> </b>

### Imports

In [25]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

### Generating data

In [13]:
series = np.arange(21)  # Generate numbers from 0 to 20
time = np.arange(len(series))  # Time index

### Preparing data

#### Train/test split

In [15]:
def train_val_split(time, series, time_step=12):

    time_train = time[:time_step]
    series_train = series[:time_step]
    time_valid = time[time_step:]
    series_valid = series[time_step:]

    return time_train, series_train, time_valid, series_valid


# Split the dataset
time_train, series_train, time_valid, series_valid = train_val_split(time, series)


#### Windowing

In [22]:
def windowed_dataset(series, 
                     window_size=3, 
                     horizon=1, 
                     batch_size=32, 
                     shuffle_buffer=10,
                     shuffle=True):
    
    # Create the dataset from the series
    ds = tf.data.Dataset.from_tensor_slices(series)
    
    # Create windows of size (window_size + horizon)
    ds = ds.window(window_size + horizon, shift=1, drop_remainder=True)
    
    # Flatten the windows into batches
    ds = ds.flat_map(lambda w: w.batch(window_size + horizon))
    
    if shuffle:
        ds = ds.shuffle(shuffle_buffer)
    
    # Split the windows into features and labels
    ds = ds.map(lambda w: (w[:-horizon], w[-horizon:]))
    
    # Batch the dataset
    ds = ds.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    return ds

# Apply the transformation to the training set and val set (horizon=1)
dataset_train = windowed_dataset(series_train)
dataset_val = windowed_dataset(series_valid, shuffle=False)

# Apply the transformation to the training set and val set (horizon=3)
dataset_train_2 = windowed_dataset(series_train, 4,3)
dataset_val_2 = windowed_dataset(series_valid, 4,3 , shuffle=False)


### Helper function for predictions

In [82]:
def forecast(model, initial_window, num_predictions, horizon=1):
    predictions = []
    current_window = initial_window.copy() 
    
    while len(predictions) < num_predictions:
        
        # Make a prediction for multiple steps ahead
        pred = model.predict(current_window[np.newaxis], verbose=0)
        print(pred.shape)
        pred_steps = pred[0][:horizon]  # Extract the predicted steps
        print(pred[0], pred[1], pred[2])

        # Add the predictions to the list, but ensure we don't exceed num_predictions
        steps_to_add = min(horizon, num_predictions - len(predictions))
        predictions.extend(pred_steps[:steps_to_add])

        # Update the window: remove the oldest values and add the new predictions
        current_window = np.roll(current_window, -steps_to_add)  # Shift left by the number of steps added
        current_window[-steps_to_add:] = pred_steps[:steps_to_add]  # Insert the new predictions at the end

    return np.array(predictions[:num_predictions])  # Return exactly num_predictions values


<a name="3"></a>
## <font color='#1f618d'> <b> 3. Metrics </font> </b>


<a name="3.1"></a>
### <font color='#2471a3'> <b> 3.1. Horizon=1 </font> </b>


First, let's create and train a model.


In [23]:
tf.random.set_seed(42)

In [27]:
# model for an horizon of 1
def build_compile_model(name, horizon=1, window_size=3):
    # Construct model
    model = tf.keras.models.Sequential([
      layers.InputLayer(input_shape=[window_size]),
      layers.Dense(128,activation='relu'),
      layers.Dense(horizon, activation="linear")                 
    ], name=name)

    # Compile model
    model.compile(loss="mse",
                    optimizer=tf.keras.optimizers.Adam(),
                    metrics=["mse"])
    
    return model


model_1 = build_compile_model_one('Simple_Dense')
model_1.summary()

Model: "Simple_Dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 128)               512       
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 641 (2.50 KB)
Trainable params: 641 (2.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
# fit
history_1 = model_1.fit(
            dataset_train,
            epochs=50,
            verbose=0,
            batch_size=32,
            validation_data=dataset_val)

In [36]:
# let's make a prediction
data = np.array([10, 11, 12])
pred = model_1.predict(data[np.newaxis])
pred

1/1 [==============================] - 0s 15ms/step


array([[14.335833]], dtype=float32)

In [37]:
pred.shape

(1, 1)

Now let's make predictions to compare with the validation set.

In [41]:
preds_1 = forecast(model_1, series_train[-3:], len(series_valid))
preds_1

array([13.078556, 14.872024, 16.600409, 18.642057, 20.911184, 23.425735,
       26.474682, 30.00109 , 34.30731 ], dtype=float32)

In [40]:
preds_1.shape

(9,)

In [132]:
# Fn. to calculate differente metrics
def evaluate_preds(y_true, y_pred):
    # Make sure float32 (for metric calculations)
    y_true = tf.cast(y_true, dtype=tf.float32)
    y_pred = tf.cast(y_pred, dtype=tf.float32)
 
    # Calculate various metrics
    mae = tf.keras.metrics.mean_absolute_error(y_true, y_pred)
    mse = tf.keras.metrics.mean_squared_error(y_true, y_pred)
    rmse = tf.sqrt(mse)
    mape = tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred)
  
    return {"mae": mae.numpy(),
          "mse": mse.numpy(),
          "rmse": rmse.numpy(),
          "mape": mape.numpy()}

In [43]:
evaluate_preds(series_valid, preds_1)

{'mae': 6.034783, 'mse': 54.18306, 'rmse': 7.360914, 'mape': 34.583866}

<a name="3.2"></a>
### <font color='#2471a3'> <b> 3.2. Horizon greater than 1 (3) </font> </b>

In [47]:
model_2 = build_compile_model_one('Simple_Dense', 3, 4) # horizon, ws
model_2.summary()

Model: "Simple_Dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 128)               640       
                                                                 
 dense_7 (Dense)             (None, 3)                 387       
                                                                 
Total params: 1027 (4.01 KB)
Trainable params: 1027 (4.01 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [48]:
# fit
history_2 = model_2.fit(
            dataset_train_2,
            epochs=50,
            verbose=0,
            batch_size=32,
            validation_data=dataset_val_2)

In [66]:
data = np.array([10, 11, 12, 20])
pred = model_2.predict(data[np.newaxis])
pred

1/1 [==============================] - 0s 15ms/step


array([[18.056765, 19.319477, 19.340502]], dtype=float32)

In [67]:
pred.shape

(1, 3)

ahora nuestra salida es (1, 3), pues nuestro horizonte es 3, probemos nuestra fn. para evaluar las métricas:

In [69]:
preds_2 = forecast(model_2, series_train[-4:], len(series_valid))

In [71]:
preds_2

array([13.111973, 14.796203, 16.523094, 18.573639, 20.847485, 23.299658,
       26.306765, 29.707779, 33.488987], dtype=float32)

In [72]:
preds_2.shape

(9,)

In [111]:
import numpy as np

def forecast(model, initial_window, num_predictions, horizon=3):
    """
    Forecasts a specified number of steps ahead using a model with a sliding window approach.

    Parameters:
    - model (tf.keras.Model): Trained time series forecasting model.
    - initial_window (np.array): Initial input sequence for forecasting.
    - num_predictions (int): Total number of forecasted points (rows) to generate.
    - horizon (int): Number of steps to predict at each iteration.

    Returns:
    - np.array: Array of predicted values with shape (num_predictions, horizon).
    """
    predictions = []
    current_window = initial_window.copy()

    for _ in range(num_predictions):
        # Make a prediction for the specified horizon
        pred = model.predict(current_window[np.newaxis], verbose=0)  # Shape: (1, horizon)
        
        # Extract predictions and reshape for output (univariate: only one feature)
        pred_steps = pred[0][:horizon]  # Take first horizon steps, shape (horizon,)

        # Append the predictions for the current step to the output
        predictions.append(pred_steps)
        
        
        
        print(f"Current window: {current_window}")
        print(f"Predictions: {pred_steps}")
        print("\n")
        

        # Update the window by removing oldest values and appending new predictions
        current_window = np.roll(current_window, -horizon)  # Shift left by horizon
        current_window[-horizon:] = pred_steps  # Insert new predictions at the end

    return np.array(predictions)  # Shape: (num_predictions, horizon)


In [125]:
preds_4 = forecast(model_2, series_train[-4:], 3) # antes 5 preds
preds_4

Current window: [ 8  9 10 11]
Predictions: [13.111973 13.012652 12.81855 ]


Current window: [11 13 13 12]
Predictions: [16.887434 16.171663 15.418688]


Current window: [12 16 16 15]
Predictions: [20.42272  19.67675  18.822645]




array([[13.111973, 13.012652, 12.81855 ],
       [16.887434, 16.171663, 15.418688],
       [20.42272 , 19.67675 , 18.822645]], dtype=float32)

In [126]:
preds_4.shape

(3, 3)

ahora tenemos 5 predicciones de tamaño 3 (predice un horizonte de 3 en cada paso)

In [114]:
evaluate_preds(series_valid, preds_4)

InvalidArgumentError: {{function_node __wrapped__Sub_device_/job:localhost/replica:0/task:0/device:CPU:0}} Incompatible shapes: [5,3] vs. [9] [Op:Sub] name: 

In [129]:
def evaluate_preds2(y_true, y_pred):
  # Make sure float32 (for metric calculations)
  y_true = tf.cast(y_true, dtype=tf.float32)
  y_pred = tf.cast(y_pred, dtype=tf.float32)

  # Calculate various metrics
  mae = tf.keras.metrics.mean_absolute_error(y_true, y_pred)
  mse = tf.keras.metrics.mean_squared_error(y_true, y_pred)
  rmse = tf.sqrt(mse)
  mape = tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred)


  # Account for different sized metrics (for longer horizons, reduce to single number)
  if mae.ndim > 0: # if mae isn't already a scalar, reduce it to one by aggregating tensors to mean
    mae = tf.reduce_mean(mae)
    mse = tf.reduce_mean(mse)
    rmse = tf.reduce_mean(rmse)
    mape = tf.reduce_mean(mape)


  return {"mae": mae.numpy(),
          "mse": mse.numpy(),
          "rmse": rmse.numpy(),
          "mape": mape.numpy()}

In [118]:
series_valid

array([12, 13, 14, 15, 16, 17, 18, 19, 20])

In [120]:
def create_y_true(val_series, window_size, horizon=3):
    y_true = []
    
    # Generamos ventanas de validación, cada una con `window_size + horizon` de largo
    for i in range(len(val_series) - window_size - horizon + 1):
        # Los valores objetivo son los siguientes `horizon` valores después de cada ventana
        true_values = val_series[i + window_size : i + window_size + horizon]
        y_true.append(true_values)
    
    return np.array(y_true)  # Esto tendrá forma (num_predictions, horizon)


"""# Ejemplo
val_series = np.arange(1, 101)  # Serie temporal de ejemplo
window_size = 10
horizon = 3

y_true = create_y_true(val_series, window_size, horizon)
print("Forma de y_true:", y_true.shape)  # Debe ser (num_predictions, horizon)
"""

Forma de y_true: (88, 3)


In [121]:
preds_4.shape

(5, 3)

In [123]:
aux  = create_y_true(series_valid, 4, 3)

In [124]:
aux.shape

(3, 3)

In [134]:
p = evaluate_preds(aux, preds_4)
p

{'mae': array([4.0189414, 1.8407383, 1.4256083], dtype=float32),
 'mse': array([17.029022 ,  5.393761 ,  2.5712423], dtype=float32),
 'rmse': array([4.1266236, 2.3224473, 1.6035094], dtype=float32),
 'mape': array([23.43033  ,  9.88953  ,  7.6360574], dtype=float32)}

valor por predicción 3

In [130]:
evaluate_preds2(aux, preds_4)

{'mae': 2.4284294, 'mse': 8.331342, 'rmse': 2.6841934, 'mape': 13.651974}